In [1]:
import requests
from bs4 import BeautifulSoup as BS
import pandas as pd
import re

In [2]:
df = pd.read_csv('movies_list_with_title.csv')

In [3]:
df

,title,link
0,Madame Web,/title/tt11057302/?ref_=sr_i_1
1,Borderlands,/title/tt4978420/?ref_=sr_i_2
2,Oppenheimer,/title/tt15398776/?ref_=sr_i_3
3,Dune: Part Two,/title/tt15239678/?ref_=sr_i_4
4,Poor Things,/title/tt14230458/?ref_=sr_i_5
...,...,...
3645,The Only Living Boy in New York,/title/tt0460890/?ref_=sr_i_3646
3646,Run All Night,/title/tt2199571/?ref_=sr_i_3647
3647,"The Adventures of Priscilla, Queen of the Desert",/title/tt0109045/?ref_=sr_i_3648
3648,Chariots of Fire,/title/tt0082158/?ref_=sr_i_3649


In [4]:
ex = '(.+)/'

pre_link = []
for link in df['link']:
    d = str(re.findall(ex,link))
    pre_link.append(d)

exc = ["[","]","'"]

c_link = []
for d in pre_link:
    x = ''.join(i for i in d if i not in exc)+'/reviews?ref_=tt_urv'   
    c_link.append(x)

In [5]:
df['c_link'] = c_link
df

,title,link,c_link
0,Madame Web,/title/tt11057302/?ref_=sr_i_1,/title/tt11057302/reviews?ref_=tt_urv
1,Borderlands,/title/tt4978420/?ref_=sr_i_2,/title/tt4978420/reviews?ref_=tt_urv
2,Oppenheimer,/title/tt15398776/?ref_=sr_i_3,/title/tt15398776/reviews?ref_=tt_urv
3,Dune: Part Two,/title/tt15239678/?ref_=sr_i_4,/title/tt15239678/reviews?ref_=tt_urv
4,Poor Things,/title/tt14230458/?ref_=sr_i_5,/title/tt14230458/reviews?ref_=tt_urv
...,...,...,...
3645,The Only Living Boy in New York,/title/tt0460890/?ref_=sr_i_3646,/title/tt0460890/reviews?ref_=tt_urv
3646,Run All Night,/title/tt2199571/?ref_=sr_i_3647,/title/tt2199571/reviews?ref_=tt_urv
3647,"The Adventures of Priscilla, Queen of the Desert",/title/tt0109045/?ref_=sr_i_3648,/title/tt0109045/reviews?ref_=tt_urv
3648,Chariots of Fire,/title/tt0082158/?ref_=sr_i_3649,/title/tt0082158/reviews?ref_=tt_urv


In [6]:
class ScrapingHelper:
     
    @staticmethod
    def get_users(soup):
        users_list = []
        users = soup.find_all('span', class_='display-name-link', limit=3)
        #for i in users:
        #    users_list.append(i.text)
        return users
    
    @staticmethod
    def get_date_review(soup):
        date_list = []
        date = soup.find_all('span', class_='review-date', limit=3)
        #for i in date:
        #    date_list.append(i.text)
        return date
    
    @staticmethod
    def get_starts(soup):
        start_list = []
        start = soup.find_all('span', class_='rating-other-user-rating', limit=3)
        #for i in start:
        #    start_list.append(re.findall('\n(\d)',i.text))
        return start
    
    @staticmethod
    def get_title_review(soup):
        title_review_list = []
        title_rev = soup.find_all('a', class_='title', limit=3)
        #for i in title_rev:
        #    title_review_list.append(i.text)
        return title_rev
    
    @staticmethod
    def get_review(soup):
        review_list = []
        review = soup.find_all('div', class_='text show-more__control', limit=3)
        #for i in review:
        #    review_list.append(i.text)
        return review

In [7]:
# movies map to store data for our dataframe
movies = {
    'user': [],
    'date_review': [],
    'stars': [],
    'title_review': [],
    'review': [],
}

In [8]:
BASE_URL = "https://imdb.com"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for _, m in df.iterrows(): 
    full_url = f"{BASE_URL}{m['c_link']}"
    data = requests.get(full_url, headers=headers)
    soup = BS(data.text, 'html.parser')
    
    user = ScrapingHelper.get_users(soup)
    date_review = ScrapingHelper.get_date_review(soup)
    stars = ScrapingHelper.get_starts(soup)
    title_review = ScrapingHelper.get_title_review(soup)
    review = ScrapingHelper.get_review(soup)
    
    for a in user:
        movies['user'].append(a.text)
    
    for b in date_review:
        movies['date_review'].append(b.text)
        
    for c in stars:
        movies['stars'].append(c.text.replace("\n",""))
    
    for d in title_review:
        movies['title_review'].append(d.text)
        
    for e in review:
        movies['review'].append(e.text)
        
    if len(movies['stars']) != len(movies['review']):
        movies['stars'].append('NaN')

In [9]:
movies = pd.DataFrame(movies)
movies.to_csv('movies_reviews_final.csv')

In [10]:
see = pd.read_csv('movies_reviews_final.csv', index_col=0)
see

,user,date_review,stars,title_review,review
0,MovieWarfare,17 February 2024,1/10,Makes Morbius look like The Dark Knight\n,I would absolutely love to be in these Sony me...
1,syntory,15 February 2024,2/10,How are bad writers able to continually get h...,Look at the profiles for the writers that wrot...
2,AlsExGal,17 February 2024,2/10,"I had a bad feeling about this one, going in....",... when I saw Dakota Johnson was starring in ...
3,Jeremy_Urquhart,20 July 2023,8/10,"A challenging watch to be sure, but a worthwh...",One of the most anticipated films of the year ...
4,Orlando_Gardner,19 July 2023,9/10,Murphy is exceptional\n,You'll have to have your wits about you and yo...
...,...,...,...,...,...
10548,trpdean,6 March 2002,9/10,"Great true story, wonderfully done\n",I watched this again last night. I had forgott...
10549,Julie-30,26 January 2003,8/10,Truly an outstanding film.\n,I beg to differ with several previous reviewer...
10550,SnoopyStyle,4 January 2014,8/10,Combustible Chemistry\n,"Violent criminal Ganz escapes from prison, and..."
10551,Prismark10,18 September 2016,8/10,The boys are back in town\n,I regard 48 Hrs as an action adventure with a ...
